In [3]:
# import necessary libraries
import pandas as pd
import sys
import tabula
import os

sys.path.append("/workspaces/acs-study/data")
#goin AWS mode rn
import boto3
b3_key = os.environ['S3_READONLY_KEY']
b3_s = os.environ['S3_READONLY_SECRET']

In [4]:
s3_client = boto3.client('s3', aws_access_key_id=b3_key, aws_secret_access_key=b3_s)
bucket_name = 'acs-study'
object_key = 'data/bernalillo_FOIA_data/2022/MAY 2022.pdf'

# Download the file
with open('filename.pdf', 'wb') as test:
    data = s3_client.download_fileobj(bucket_name, object_key, test)
    
columns = ['date', 'time', 'inc_num', 'report', 'pri', 'code', 'code_description', 'area', 'sect', 'beat', 'dispo', 'address', 'city', 'zip', 'lat', 'long', 'whack']
temp_columns = columns[:-1]

tables = tabula.read_pdf('filename.pdf', pages='all')
print('success reading')

Oct 05, 2023 6:15:35 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Oct 05, 2023 6:15:35 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Oct 05, 2023 6:15:35 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>


success reading


In [20]:
df = tables[0]
df.columns = columns

for i in range(len(tables)):
    if i == 0:
        continue
    temp_df = tables[i]
    temp_df.columns = temp_columns
    df = pd.concat([df, temp_df], ignore_index=True)
    
df = df.drop(['whack'], axis=1)

In [55]:
import folium
locations = df[['lat', 'long', 'code_description']].dropna()
map = folium.Map(location=[locations.lat.mean(), locations.long.mean()], zoom_start=6, control_scale=True)
for index, location in locations.sample(2000).iterrows():
    folium.Marker([location['lat'], location['long']], popup=location['code_description']).add_to(map)

In [56]:
map